In [1]:
'''After doing alterations in MYSLQ workbench'''
# Load the master_analytics_data.csv file into a DataFrame
import pandas as pd
try:
    df = pd.read_csv('/content/master_analytics_data.csv')
except FileNotFoundError:
    print("Error: The file 'master_analytics_data.csv' was not found. Please ensure it's in the correct directory.")
    exit()
print(df.head())

   student_id signup_date              course_name  source_campaign_id  \
0        2000  2024-01-01  Python For Data Science                1023   
1        2002  2024-01-08  Python For Data Science                1000   
2        2003  2024-01-05  Python For Data Science                1025   
3        2013  2024-01-15       Data Analytics 101                1027   
4        2022  2024-01-28  Machine Learning Basics                1019   

   completion_percentage   platform  ad_spend  clicks  impressions  
0                  27.88   LinkedIn  45267.23    3842        46104  
1                  75.61  Instagram  21854.31    1360        25840  
2                  93.37   LinkedIn  48252.74     837        15903  
3                  19.98   LinkedIn  17642.05     879        14943  
4                  29.66    YouTube  45919.42    2455        14730  


In [2]:
# Group by campaign to get accurate total ad spend for each campaign
campaign_summary = df.groupby('source_campaign_id').agg(
    total_ad_spend=('ad_spend', 'first') # Use 'first' since ad_spend should be the same for all rows from the same campaign
).reset_index()

# Calculate total signups from the entire dataset
total_signups = df['student_id'].nunique()

# Get the total ad spend from the campaign summary
total_ad_spend = campaign_summary['total_ad_spend'].sum()

# Calculate Cost Per Signup (CPS)
cps = total_ad_spend / total_signups
print(f"Cost Per Signup (CPS): ${cps:.2f}")

# Calculate the number of completed students (completion > 90%)
completed_students = df[df['completion_percentage'] > 90]['student_id'].nunique()

# Calculate Cost Per Completed Student (CPCS) with error handling
if completed_students > 0:
    cpcs = total_ad_spend / completed_students
    print(f"Cost Per Completed Student (CPCS): ${cpcs:.2f}")
else:
    print("No students completed more than 90% of a course, so CPCS cannot be calculated.")

Cost Per Signup (CPS): $3543.95
Cost Per Completed Student (CPCS): $25313.95


In [3]:
import pandas as pd
import numpy as np

# Assume 'df' is your DataFrame loaded from 'master_analytics_data.csv'
# And that you have already cleaned the column names.

# Step 1: Calculate campaign-level metrics
# The previous code already calculated CPS and CPCS. Let's make sure our campaign_metrics table has all the data we need for the next step.
campaign_metrics = df.groupby('source_campaign_id').agg(
    ad_spend=('ad_spend', 'first'),
    platform=('platform', 'first'),
    total_signups=('student_id', 'count'),
    completed_students=('completion_percentage', lambda x: (x > 90).sum())
).reset_index()

# Calculate CPS and CPCS for each campaign
campaign_metrics['CPS'] = campaign_metrics['ad_spend'] / campaign_metrics['total_signups']
campaign_metrics['CPCS'] = campaign_metrics['ad_spend'] / campaign_metrics['completed_students']

# Fill any NaN values (from division by zero) with 0 for cleaner output
campaign_metrics.fillna(0, inplace=True)

# Step 2: Group and compare by platform
# Now, group the campaign-level metrics by the 'platform' column.
# We will calculate both the mean and the standard deviation for the cost metrics.
platform_summary = campaign_metrics.groupby('platform').agg(
    average_cps=('CPS', 'mean'),
    std_dev_cps=('CPS', 'std'),
    average_cpcs=('CPCS', 'mean'),
    total_ad_spend=('ad_spend', 'sum')
).reset_index()

# Also, calculate the average and standard deviation of completion percentage
# from the original DataFrame for each platform.
completion_summary = df.groupby('platform').agg(
    average_completion_percentage=('completion_percentage', 'mean'),
    std_dev_completion_percentage=('completion_percentage', 'std')
).reset_index()

# Merge the two summaries for a complete view
final_summary = pd.merge(platform_summary, completion_summary, on='platform')

# Display the final summary table
print("\nPlatform-level Performance Summary:")
print(final_summary)



Platform-level Performance Summary:
    platform  average_cps  std_dev_cps  average_cpcs  total_ad_spend  \
0  Instagram  4577.322006  4913.121001           inf       192817.08   
1   LinkedIn  4172.811266  2762.738905           inf       273447.22   
2    YouTube  4515.266447  3811.572294           inf       242526.21   

   average_completion_percentage  std_dev_completion_percentage  
0                      58.140000                      26.198574  
1                      56.303514                      25.436750  
2                      50.561429                      27.237625  


In [4]:
final_summary.to_csv('Final_BDA_Project.csv', index=False)
print("\nSuccess! The final summary has been exported to 'platform_performance_summary.csv'.")


Success! The final summary has been exported to 'platform_performance_summary.csv'.
